In [1]:
from transformers import AutoTokenizer, pipeline
from optimum.intel import OVModelForCausalLM, OVWeightQuantizationConfig
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from optimum.exporters.openvino import export_from_model
import os
import torch

In [6]:
model_id = 'llm'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id
  )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
save_directory = "llm"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('llm\\tokenizer_config.json',
 'llm\\special_tokens_map.json',
 'llm\\vocab.json',
 'llm\\merges.txt',
 'llm\\added_tokens.json',
 'llm\\tokenizer.json')

In [ ]:
model_id = 'Qwen/Qwen2.5-1.5B-Instruct'

quantization_config = OVWeightQuantizationConfig(bits=8)
model = AutoModelForCausalLM.from_pretrained(model_id)
export_from_model(model, output="llm_quantized", task="text-generation-with-past")

tokenizer = AutoTokenizer.from_pretrained(model_id)

Can't determine type of OV quantization config. Please specify explicitly whether you intend to run weight-only quantization or not with `weight_only` parameter. Creating an instance of OVWeightQuantizationConfig.
The model weights will be quantized to int8.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using framework PyTorch: 2.4.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:1116: TracerWarning: Con

INFO:nncf:Statistics of the bitwidth distribution:
+----------------+-----------------------------+----------------------------------------+
|   Num bits (N) | % all parameters (layers)   | % ratio-defining parameters (layers)   |
+================+=============================+========================================+
|              8 | 100% (197 / 197)            | 100% (197 / 197)                       |
+----------------+-----------------------------+----------------------------------------+


Output()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model_id = "srang992/Qwen-2.5-3B-Instruct-ov-INT4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = OVModelForCausalLM.from_pretrained(
    model_id, device='gpu'
)

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

openvino_model.bin:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

openvino_model.xml:   0%|          | 0.00/4.34M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

Compiling the model to GPU ...
Setting OpenVINO CACHE_DIR to C:\Users\Jandrik\.cache\huggingface\hub\models--srang992--Qwen-2.5-3B-Instruct-ov-INT4\snapshots\a48c0ed06f0b406b857491bb566f744473b3810a\model_cache


RuntimeError: Exception from src/inference/src/cpp/core.cpp:107:
Exception from src/inference/src/dev/plugin.cpp:54:
Exception from src/common/transformations/src/transformations/convert_precision.cpp:1340:
Precision conversion from bf16 to f16 is not supported




In [39]:
from transformers import TextStreamer
ocr_text = """345
D483X
DO Not ACCEPT
IF SEAL
100 TABLETS
IS BROKEN
Allopurinol
Llanole 100 mg Tablet
Antigout
Manufactured by AMHERST LABORATORIES, INC.
UNILAB Pharma Campus, Barangay Mamplasan
For JNLagunacPhilippines
R
No. 66 United Street, Mandaluyong City; Metro Manila, Philippines"""
inst = f"""Extract the medication name from this OCR text of a medicine package:
{ocr_text} 

Return only the generic medication name, manufacturer/laboratory, dosage, and packaging of the OCR text, don't add anything else like addresses. Return them seperated by spaces.
"""

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": inst},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_text=tokenizer.apply_chat_template(messages, tokenize=False)

# Tokenize the sample
inputs = tokenizer([input_text], return_tensors='pt')

# Call generate on the inputs
out = model.generate(
    **inputs,
    max_new_tokens=96,
    streamer=TextStreamer(tokenizer=tokenizer, skip_special_tokens=True),
    pad_token_id=tokenizer.eos_token_id,
    #temperature=0.7,
    do_sample=False
)

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Extract the medication name from this OCR text of a medicine package:
345
D483X
DO Not ACCEPT
IF SEAL
100 TABLETS
IS BROKEN
Allopurinol
Llanole 100 mg Tablet
Antigout
Manufactured by AMHERST LABORATORIES, INC.
UNILAB Pharma Campus, Barangay Mamplasan
For JNLagunacPhilippines
R
No. 66 United Street, Mandaluyong City; Metro Manila, Philippines 

Return only the generic medication name, manufacturer/laboratory, dosage, and packaging of the OCR text, don't add anything else like addresses. Return them seperated by spaces.



c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should s

system
Allopurinol Llanole 100 mg Tablet Amherst Laboratories, Inc. Allopurinol


In [24]:
result

'DONOTACCEPT IF SEAL 100 TABLETS IS BROKEN Allopurinol Llanol@ 100 mg Tablet Antigout R Manufactured by AMHERST LABORATORIES, INC. UNILAB Pharma Campus, Barangay Mamplasan Binan Laguna Philippines for UNILAB, Inc. No.66 United Street Mandaluyong CityMetro ManilaPhilippines'

In [54]:
extracted_query = tokenizer.batch_decode(out)[0]
extracted_query = extracted_query[extracted_query.index('<|im_end|>\n<|im_start|>system\n')+len('<|im_end|>\n<|im_start|>system\n'):]
extracted_query = extracted_query.replace('<|im_end|>', '')
extracted_query

'Allopurinol Llanole 100 mg Tablet Amherst Laboratories, Inc. Allopurinol'

### Test

In [ ]:
import archived.ocr_postprocess as ocr_postprocess

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Model... 




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
extracted_result = ocr_postprocess.extract_info_from_ocr(result)

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Extract the medication name from this OCR text of a medicine package:
    DONOTACCEPT IF SEAL 100 TABLETS IS BROKEN Allopurinol Llanol@ 100 mg Tablet Antigout R Manufactured by AMHERST LABORATORIES, INC. UNILAB Pharma Campus, Barangay Mamplasan Binan Laguna Philippines for UNILAB, Inc. No.66 United Street Mandaluyong CityMetro ManilaPhilippines 

    Return only the generic medication name, manufacturer/laboratory, dosage, and packaging of the OCR text, don't add anything else like addresses. Return them seperated by spaces.
system
Allopurinol 100 mg Tablet AMHERST LABORATORIES, INC. Unilab Pharma Campus, Barangay Mamplasan Binan Laguna Philippines


## Matching

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd

<function fuzzywuzzy.fuzz.partial_token_sort_ratio(s1, s2, force_ascii=True, full_process=True)>

In [19]:
fda_df = pd.read_csv('FDA_ALL.csv')
rx_df = pd.read_csv('RX_ALL.csv')

def match_with_fda(match_string):
    matches = []
    scores = []

    for i in range(len(fda_df)):
        #print(fda_df.iloc[i]['Brand Name'])
        
        medication_entry = str(fda_df.iloc[i]['Generic Name']) + str(fda_df.iloc[i]['Brand Name']) + str(fda_df.iloc[i]['Manufacturer']) + str(fda_df.iloc[i]['Dosage Strength'])  + str(fda_df.iloc[i]['Packaging'])
        score = fuzz.ratio(medication_entry.lower(), match_string.lower())
        matches.append(fda_df.iloc[i]['INDEX'])
        scores.append(score)
    
    matches, scores = zip(*sorted(zip(matches, scores), key=lambda x: x[1], reverse=True))
    return matches, scores

def match_with_rx(match_string):
    matches = []
    scores = []

    for i in range(len(rx_df)):
        
        medication_entry = str(rx_df.iloc[i]['Drug_Name'])
        score = fuzz.token_sort_ratio(medication_entry.lower(), match_string.lower())
        matches.append(rx_df.iloc[i]['INDEX'])
        scores.append(score)
    
    matches, scores = zip(*sorted(zip(matches, scores), key=lambda x: x[1], reverse=True))
    return matches, scores

In [22]:
extracted_result

'Allopurinol 100 mg Tablet AMHERST LABORATORIES, INC. Unilab Pharma Campus, Barangay Mamplasan Binan Laguna Philippines'

In [20]:
import medication_matching

In [21]:
medication_matching.get_info(extracted_result)

[{'INDEX': 32,
  'Registration Number': 'DR-7571',
  'Generic Name': 'Allopurinol',
  'Brand Name': 'Llanol',
  'Dosage Strength': '100 mg',
  'Dosage Form': 'Tablet',
  'Classification': 'Prescription Drug (Rx)',
  'Packaging': "Aluminum Foil Strip x 4's",
  'Pharmacologic Category': 'Antigout',
  'Manufacturer': 'Amherst laboratories, Inc.',
  'Country of Origin': 'Philippines',
  'Trader': 'UNILAB, Inc.',
  'Importer': nan,
  'Distributor': nan,
  'Application Type': 'Renewal',
  'Issuance Date': '26-May-22',
  'Expiry Date': '30-May-27',
  'match_score': 55,
  'rx_info': {'INDEX': 313,
   'Drug_Name': 'Allopurinol (Zyloprim)',
   'URL': 'https://www.rxlist.com/zyloprim-drug.htm'}},
 {'INDEX': 67,
  'Registration Number': 'DR-X7781',
  'Generic Name': 'Allopurinol',
  'Brand Name': 'Purinase',
  'Dosage Strength': '300 mg',
  'Dosage Form': 'Tablet',
  'Classification': 'Prescription Drug (RX)',
  'Packaging': "Foil strip x 10's (Box of 50's)",
  'Pharmacologic Category': '-',
  'Ma

In [80]:
m, s = match_with_rx('Allopurinol')
print(m[:5])
print(s[:5])
rx_df.iloc[m[0]]

(313, 8772, 317, 3904, 5419)
(71, 71, 55, 55, 55)


INDEX                                             313
Drug_Name                      Allopurinol (Zyloprim)
URL          https://www.rxlist.com/zyloprim-drug.htm
Name: 313, dtype: object

In [73]:
a = """Oeja
UNILAB
60 mL
Ambroxol Hci]
Expel
15 mg/S mL Syrup
Mucolytic
Manufactured by AMHERST LABORATORIES; INC.
UNILAB Pharma
ampus; Barangay Mamplasan
Binan; Laguna, Philippines
for UNILAB, Inc:
No. 66 United Street; Mandaluyong
Metro Manila, Philippines
City"""
m, s = match_with_fda("""Allopurinol Llanole 100 mg Tablet Amherst Laboratories, Inc. Allopurinol""")

In [74]:
print(m[:5])
print(s[:5])
fda_df.iloc[m[0]]

(32, 69, 2498, 2503, 9821)
(70, 63, 62, 59, 59)


INDEX                                             32
Registration Number                          DR-7571
Generic Name                             Allopurinol
Brand Name                                    Llanol
Dosage Strength                               100 mg
Dosage Form                                   Tablet
Classification                Prescription Drug (Rx)
Packaging                  Aluminum Foil Strip x 4's
Pharmacologic Category                      Antigout
Manufacturer              Amherst laboratories, Inc.
Country of Origin                        Philippines
Trader                                  UNILAB, Inc.
Importer                                         NaN
Distributor                                      NaN
Application Type                             Renewal
Issuance Date                              26-May-22
Expiry Date                                30-May-27
Name: 32, dtype: object

In [77]:
match_entry = dict(fda_df.iloc[m[0]])
match_entry['match_score'] = s[0]

In [79]:
match_entry

{'INDEX': 32,
 'Registration Number': 'DR-7571',
 'Generic Name': 'Allopurinol',
 'Brand Name': 'Llanol',
 'Dosage Strength': '100 mg',
 'Dosage Form': 'Tablet',
 'Classification': 'Prescription Drug (Rx)',
 'Packaging': "Aluminum Foil Strip x 4's",
 'Pharmacologic Category': 'Antigout',
 'Manufacturer': 'Amherst laboratories, Inc.',
 'Country of Origin': 'Philippines',
 'Trader': 'UNILAB, Inc.',
 'Importer': nan,
 'Distributor': nan,
 'Application Type': 'Renewal',
 'Issuance Date': '26-May-22',
 'Expiry Date': '30-May-27',
 'match_score': 70}

## Easy OCR

In [15]:
from PIL import Image
import easyocr

In [19]:
reader = easyocr.Reader(['en'], detector='craft')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [56]:
' '.join([i[-2] for i in result])

'345 D483X DO Not ACCEPT IF SEAL 100 TABLETS IS BROKEN Allopurinol Llanole 100 mg Tablet Antigout Manufactured by AMHERST LABORATORIES, INC. UNILAB Pharma Campus, Barangay Mamplasan For JNLagunacPhilippines R No. 66 United Street, Mandaluyong City; Metro Manila, Philippines'

In [18]:
IMAGE_PATH = 'test.jpg'
result = reader.readtext(IMAGE_PATH)
for i in result:
    print(i[-2])

345
D483X
DO Not ACCEPT
IF SEAL
100 TABLETS
IS BROKEN
Allopurinol
Llanole 100 mg Tablet
Antigout
Manufactured by AMHERST LABORATORIES, INC.
UNILAB Pharma Campus, Barangay Mamplasan
For JNLagunacPhilippines
R
No. 66 United Street, Mandaluyong City; Metro Manila, Philippines


## PaddleOCR

In [2]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to C:\Users\Jandrik/.paddleocr/whl\det\en\en_PP-OCRv3_det_infer\en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:15<00:00, 258.32it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to C:\Users\Jandrik/.paddleocr/whl\rec\en\en_PP-OCRv4_rec_infer\en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:01<00:00, 5458.58it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\Jandrik/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:12<00:00, 171.60it/s]

[2025/02/19 10:31:11] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

In [11]:
import ocr_module

[2025/02/19 10:40:05] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

In [13]:
result = ocr_module.extract_image_text('test.jpg')

[2025/02/19 10:41:13] ppocr DEBUG: dt_boxes num : 13, elapsed : 0.09532475471496582
[2025/02/19 10:41:13] ppocr DEBUG: cls num  : 13, elapsed : 0.031423330307006836
[2025/02/19 10:41:13] ppocr DEBUG: rec_res num  : 13, elapsed : 0.789665937423706


In [9]:
result[1]

IndexError: list index out of range